In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv("sqm/bokaro.csv")




In [64]:
df 

,Location,Description,Total_Inspection_Completed_Works,Completed_Works_U%,Ongoing_Works,Ongoing_U%,Maintenance_Works,Maintenance_U%,Bridge_Works,Bridge_U%,Total_Inspection
0,Chandankiyari,Unsatisfactory works grading during 2...,4,0,0.00%,0,0.00%,0,0.00%,0,0.00%
1,Chandankiyari,Unsatisfactory works grading during 2...,6,0,0.00%,0,0.00%,0,0.00%,0,0.00%
2,Chas,Unsatisfactory works grading during 2...,4,0,0.00%,0,0.00%,0,0.00%,0,0.00%
3,Chas,Unsatisfactory works grading during 2...,5,0,0.00%,0,0.00%,0,0.00%,0,0.00%
4,Gomia,Unsatisfactory works grading during 2...,12,0,0.00%,0,0.00%,0,0.00%,0,0.00%
5,Gomia,Unsatisfactory works grading during 2...,34,0,0.00%,0,0.00%,1,2.94%,0,0.00%
6,Jaridhi,Unsatisfactory works grading during 2...,0,0,0.00%,0,0.00%,0,0.00%,0,0.00%
7,Jaridhi,Unsatisfactory works grading during 2...,7,0,0.00%,0,0.00%,0,0.00%,0,0.00%
8,Kasmar,Unsatisfactory works grading during 2...,3,0,0.00%,0,0.00%,0,0.00%,0,0.00%
9,Kasmar,Unsatisfactory works grading during 2...,4,0,0.00%,0,0.00%,0,0.00%,0,0.00%


In [65]:
import pandas as pd

# Assuming df is your DataFrame
# Check and convert 'Total_Inspection_Completed_Works'
if pd.api.types.is_numeric_dtype(df['Total_Inspection_Completed_Works']):
    df['Total_Inspection_Completed_Works'] = df['Total_Inspection_Completed_Works'].astype(float)
else:
    df['Total_Inspection_Completed_Works'] = df['Total_Inspection_Completed_Works'].str.replace(',', '').astype(float)

# List of columns to convert to numeric after removing '%'
percentage_columns = ['Completed_Works_U%', 'Ongoing_Works', 'Ongoing_U%', 'Maintenance_Works', 'Maintenance_U%', 'Bridge_Works', 'Bridge_U%']

# Loop through the columns and convert to numeric after removing '%'
for col in percentage_columns:
    # Check if the column is already numeric
    if not pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].str.rstrip('%').astype('float') / 100.0

# Print the data types after conversion
print(df.dtypes)


Location                             object
Description                          object
Total_Inspection_Completed_Works    float64
Completed_Works_U%                    int64
Ongoing_Works                       float64
Ongoing_U%                            int64
Maintenance_Works                   float64
Maintenance_U%                        int64
Bridge_Works                        float64
Bridge_U%                             int64
Total_Inspection                     object
dtype: object


In [66]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
import plotly.express as px

# Assuming df is your DataFrame

# Convert 'Total_Inspection' to numeric
df['Total_Inspection'] = pd.to_numeric(df['Total_Inspection'].str.replace(',', ''), errors='coerce')

# Check and convert 'Total_Inspection_Completed_Works'
if pd.api.types.is_numeric_dtype(df['Total_Inspection_Completed_Works']):
    df['Total_Inspection_Completed_Works'] = df['Total_Inspection_Completed_Works'].astype(float)
else:
    df['Total_Inspection_Completed_Works'] = df['Total_Inspection_Completed_Works'].str.replace(',', '').astype(float)

# List of columns to convert to numeric after removing '%'
percentage_columns = ['Completed_Works_U%', 'Ongoing_Works', 'Ongoing_U%', 'Maintenance_Works', 'Maintenance_U%', 'Bridge_Works', 'Bridge_U%']

# Loop through the columns and convert to numeric after removing '%'
for col in percentage_columns:
    # Check if the column is already numeric
    if not pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].str.rstrip('%').astype('float') / 100.0

# Identify numeric columns
numeric_columns = df.select_dtypes(include=['number']).columns

# Impute missing values for each numeric column separately
imputer = SimpleImputer(strategy='mean')
for col in numeric_columns:
    df[col] = imputer.fit_transform(df[[col]])

# Isolation Forest
features = df[['Total_Inspection_Completed_Works', 'Completed_Works_U%', 'Ongoing_Works', 'Ongoing_U%',
                      'Maintenance_Works', 'Maintenance_U%', 'Bridge_Works', 'Bridge_U%', 'Total_Inspection']]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

iso_forest = IsolationForest(random_state=42, contamination=0.05)
df['outlier'] = iso_forest.fit_predict(features_scaled)

# Accuracy Score
true_labels = [1 if x == 1 else -1 for x in df['outlier']]
predicted_labels = iso_forest.predict(features_scaled)
accuracy = accuracy_score(true_labels, predicted_labels)

print(f'Isolation Forest Accuracy: {accuracy}')

# Visualization using Plotly
fig = px.scatter_3d(df, x='Total_Inspection_Completed_Works', y='Ongoing_Works', z='Maintenance_Works',
                    color='outlier', title='Isolation Forest Outliers',
                    labels={'Total_Inspection_Completed_Works': 'Completed Works',
                            'Ongoing_Works': 'Ongoing Works',
                            'Maintenance_Works': 'Maintenance Works'})

fig.update_layout(scene=dict(aspectmode="cube"))
fig.show()


/Users/pranaymishra/Desktop/care/.conda/lib/python3.10/site-packages/sklearn/impute/_base.py:558: UserWarning:

Skipping features without any observed values: ['Total_Inspection']. At least one non-missing value is needed for imputation with strategy='mean'.



Isolation Forest Accuracy: 1.0
